<a href="https://colab.research.google.com/github/divyanshsaxena21/X-Ray_Analysis/blob/main/X_RAY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!kaggle datasets download jtiptj/chest-xray-pneumoniacovid19tuberculosis

Dataset URL: https://www.kaggle.com/datasets/jtiptj/chest-xray-pneumoniacovid19tuberculosis
License(s): other
 99% 1.72G/1.74G [00:09<00:00, 240MB/s]
100% 1.74G/1.74G [00:09<00:00, 201MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/chest-xray-pneumoniacovid19tuberculosis.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt


In [ ]:
# Define paths to your data directories
train_dir = '/content/train'
test_dir = '/content/test'
val_dir = '/content/val'

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150), # Adjust as needed
    batch_size=32, # Adjust as needed
    class_mode='categorical' # For multi-class classification
)

validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False # Important for evaluation
)


Found 6326 images belonging to 4 classes.
Found 38 images belonging to 4 classes.
Found 771 images belonging to 4 classes.


In [ ]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5)) # Add dropout for regularization
model.add(Dense(4, activation='softmax')) # 4 classes: COVID-19, Normal, Pneumonia, Tuberculosis


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10
197/197 [==============================] - 92s 464ms/step - loss: 0.7021 - accuracy: 0.7302 - val_loss: 0.7364 - val_accuracy: 0.6875
Epoch 2/10
197/197 [==============================] - 90s 455ms/step - loss: 0.4483 - accuracy: 0.8314 - val_loss: 0.8247 - val_accuracy: 0.6562
Epoch 3/10
197/197 [==============================] - 88s 448ms/step - loss: 0.3872 - accuracy: 0.8564 - val_loss: 0.7718 - val_accuracy: 0.7500
Epoch 4/10
197/197 [==============================] - 87s 441ms/step - loss: 0.3439 - accuracy: 0.8726 - val_loss: 0.5513 - val_accuracy: 0.8125
Epoch 5/10
197/197 [==============================] - 88s 446ms/step - loss: 0.3208 - accuracy: 0.8807 - val_loss: 0.5344 - val_accuracy: 0.8125
Epoch 6/10
197/197 [==============================] - 87s 443ms/step - loss: 0.3229 - accuracy: 0.8824 - val_loss: 0.6452 - val_accuracy: 0.8125
Epoch 7/10
197/197 [==============================] - 88s 444ms/step - loss: 0.2896 - accuracy: 0.8947 - val_loss: 0.4324 - val_ac

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('Test accuracy:', test_acc)

24/24 [==============================] - 7s 259ms/step - loss: 0.4147 - accuracy: 0.8620
Test accuracy: 0.8619791865348816


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to predict the class of a given image
def predict_image(img_path, model):
    # Step 1: Load the image and resize it to (150, 150)
    img = image.load_img(img_path, target_size=(150, 150))  # Load the image
    img_array = image.img_to_array(img)  # Convert image to numpy array
    img_array = img_array / 255.0  # Rescale the pixel values to [0, 1]

    # Step 2: Expand the dimensions to create a batch of 1 image (model expects batch input)
    img_array = np.expand_dims(img_array, axis=0)

    # Step 3: Make the prediction
    predictions = model.predict(img_array)

    # Step 4: Get the class with the highest probability
    class_idx = np.argmax(predictions, axis=1)

    # Class names (based on the order in your directories)
    class_labels = ['COVID-19', 'Normal', 'Pneumonia', 'Tuberculosis']

    # Step 5: Return the predicted class label
    return class_labels[class_idx[0]]

# Example usage:
img_path = input('path_to_your_image.jpg')  # Replace with the path to the image you want to predict
predicted_class = predict_image(img_path, model)
print(f"The predicted class is: {predicted_class}")


path_to_your_image.jpg/content/val/PNEUMONIA/person1946_bacteria_4874.jpeg
1/1 [==============================] - 0s 97ms/step
The predicted class is: Pneumonia


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model('/content/model.h5')

In [ ]:
def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions, axis=1)
    class_labels = ['COVID-19', 'Normal', 'Pneumonia', 'Tuberculosis']
    return class_labels[class_idx[0]]

In [ ]:
img_path = input("Enter path of image") # Replace with path to your test image
predicted_class = predict_image(img_path, model)
print(f"The predicted class is: {predicted_class}")

Enter path of image/content/val/COVID19/COVID19(567).jpg
1/1 [==============================] - 0s 80ms/step
The predicted class is: COVID-19
